In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200823.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,8/19/2020 20:32:37,Spanking the Monkey,The Royal Tenenbaums,Samurai Rebellion,NaN,NaN,THIS JOB IS KILLING ME,BAD HISTORY,SURREALITY BITES,EASTERN EUROPEAN,NaN,NaN
1,8/19/2020 20:35:16,Samurai Rebellion,Spanking the Monkey,The Sacrifice,The Royal Tenenbaums,Festen,THIS JOB IS KILLING ME,BAD HISTORY,EASTERN EUROPEAN,SURREALITY BITES,THRILLER? I BARELY KNOW 'ER,No Theme/Free-for-all
2,8/19/2020 20:41:40,The Sacrifice,Samurai Rebellion,Festen,Spanking the Monkey,NaN,SURREALITY BITES,EASTERN EUROPEAN,THRILLER? I BARELY KNOW 'ER,THIS JOB IS KILLING ME,BAD HISTORY,No Theme/Free-for-all
3,8/20/2020 9:04:58,Samurai Rebellion,The Royal Tenenbaums,Spanking the Monkey,The Sacrifice,Festen,THIS JOB IS KILLING ME,BAD HISTORY,EASTERN EUROPEAN,THRILLER? I BARELY KNOW 'ER,SURREALITY BITES,No Theme/Free-for-all
4,8/20/2020 9:05:44,Festen,The Sacrifice,The Royal Tenenbaums,Samurai Rebellion,Spanking the Monkey,EASTERN EUROPEAN,THIS JOB IS KILLING ME,THRILLER? I BARELY KNOW 'ER,SURREALITY BITES,BAD HISTORY,NaN
5,8/20/2020 19:53:03,Samurai Rebellion,The Royal Tenenbaums,Festen,Spanking the Monkey,NaN,SURREALITY BITES,BAD HISTORY,EASTERN EUROPEAN,THRILLER? I BARELY KNOW 'ER,THIS JOB IS KILLING ME,NaN
6,8/21/2020 9:24:55,The Sacrifice,Samurai Rebellion,Spanking the Monkey,Festen,The Royal Tenenbaums,THIS JOB IS KILLING ME,THRILLER? I BARELY KNOW 'ER,EASTERN EUROPEAN,BAD HISTORY,SURREALITY BITES,NaN
7,8/21/2020 9:40:53,Samurai Rebellion,The Sacrifice,Spanking the Monkey,Festen,The Royal Tenenbaums,THIS JOB IS KILLING ME,EASTERN EUROPEAN,SURREALITY BITES,THRILLER? I BARELY KNOW 'ER,BAD HISTORY,No Theme/Free-for-all
8,8/21/2020 15:58:14,The Royal Tenenbaums,The Sacrifice,Spanking the Monkey,Samurai Rebellion,Festen,BAD HISTORY,THIS JOB IS KILLING ME,THRILLER? I BARELY KNOW 'ER,SURREALITY BITES,EASTERN EUROPEAN,No Theme/Free-for-all


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Samurai Rebellion', 'Spanking the Monkey', 'The Sacrifice',
       'The Royal Tenenbaums', 'Festen'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Samurai Rebellion': <Candidate('Samurai Rebellion')>,
 'Spanking the Monkey': <Candidate('Spanking the Monkey')>,
 'The Sacrifice': <Candidate('The Sacrifice')>,
 'The Royal Tenenbaums': <Candidate('The Royal Tenenbaums')>,
 'Festen': <Candidate('Festen')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Spanking the Monkey, The Royal Tenenbaums, Samurai Rebellion)>,
 <Ballot(Samurai Rebellion, Spanking the Monkey, The Sacrifice, The Royal Tenenbaums, Festen)>,
 <Ballot(The Sacrifice, Samurai Rebellion, Festen, Spanking the Monkey)>,
 <Ballot(Samurai Rebellion, The Royal Tenenbaums, Spanking the Monkey, The Sacrifice, Festen)>,
 <Ballot(Festen, The Sacrifice, The Royal Tenenbaums, Samurai Rebellion, Spanking the Monkey)>,
 <Ballot(Samurai Rebellion, The Royal Tenenbaums, Festen, Spanking the Monkey)>,
 <Ballot(The Sacrifice, Samurai Rebellion, Spanking the Monkey, Festen, The Royal Tenenbaums)>,
 <Ballot(Samurai Rebellion, The Sacrifice, Spanking the Monkey, Festen, The Royal Tenenbaums)>,
 <Ballot(The Royal Tenenbaums, The Sacrifice, Spanking the Monkey, Samurai Rebellion, Festen)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate               Votes  Status
--------------------  -------  --------
Samurai Rebellion           4  Hopeful
The Sacrifice               2  Hopeful
The Royal Tenenbaums        1  Hopeful
Spanking the Monkey         1  Hopeful
Festen                      1  Rejected

ROUND 2
Candidate               Votes  Status
--------------------  -------  --------
Samurai Rebellion           4  Hopeful
The Sacrifice               3  Hopeful
The Royal Tenenbaums        1  Rejected
Spanking the Monkey         1  Rejected
Festen                      0  Rejected

FINAL RESULT
Candidate               Votes  Status
--------------------  -------  --------
Samurai Rebellion           5  Elected
The Sacrifice               4  Rejected
The Royal Tenenbaums        0  Rejected
Spanking the Monkey         0  Rejected
Festen                      0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
THIS JOB IS KILLING ME             5  Elected
SURREALITY BITES                   2  Rejected
BAD HISTORY                        1  Rejected
EASTERN EUROPEAN                   1  Rejected
THRILLER? I BARELY KNOW 'ER        0  Rejected
No Theme/Free-for-all              0  Rejected

